# Möbius‑Digit‑Thread Scanner (v5)

Searches each Möbius‑breath output for **digit runs that coincide with famous constants** (π, φ, e, Euler–Mascheroni γ, Feigenbaum δ) **and powers of 3** (27, 81, 127, 729 …).  
A “hit” = ≥ 4 consecutive matching digits after the decimal.

**Workflow inside this notebook**

1. **Parameter generator**  
   * φ_scale grid around 0.003943 … (±6 × 10⁻⁷)  
   * tiny exponents k = 4, 4.5  
   * cycles = 1 (best so far)

2. **Run one breath** → record value `x`.

3. **Digit‑thread scan**  
   * Strip “0.”, keep first 300 digits of `x`.  
   * For each target constant (π, φ, e, γ, δ) grab its first 500 digits and mark substrings of length ≥ 4 shared with `x`.  
   * Check whether `rel_err < 0.001" and whether digit thread exists.

4. **Power‑of‑3 proximity**  
   * Compare `x` and `x⁻¹` to {9, 27, 81, 127, 729} and highlight ratios within 1 %.

Outputs a table of all runs with: parameters, rel_err, digit‑thread(s), 3‑power proximity flag.

In [1]:
import mpmath as mp, pandas as pd, itertools, re, math
mp.mp.dps = 350
ALPHA_INV = mp.mpf('137.035999084'); ALPHA = 1/ALPHA_INV; PHI = (1+mp.sqrt(5))/2
PI = mp.pi; E = mp.e; GAMMA = mp.euler; DELTA = mp.mpf('4.669201609')
def const_digits(c, n=500):
    s=str(c)[2:]  # skip '3.'
    if len(s)<n: s += '0'*(n-len(s))
    return s[:n]
DIGIT_BANK = {
    'pi':   const_digits(PI),
    'phi':  const_digits(PHI),
    'e':    const_digits(E),
    'gamma':const_digits(GAMMA),
    'delta':const_digits(DELTA)}


In [2]:
def fib_word_bits(n):
    a,b='0','01'
    while len(b)<n: a,b=b,b+a
    return b[:n]
def fib_fraction(bits):
    return mp.fsum(int(b)*mp.mpf(2)**(-(k+1)) for k,b in enumerate(fib_word_bits(bits)))


In [3]:
def breath_shift(x,tiny,phi_scale):
    x = phi_scale*PHI / x
    x = x - tiny
    x = 1/x / 9
    x = 1/x
    x = x / 9
    return x

## Scan parameter grid and search digit threads

In [4]:
def digit_threads(xstr):
    hits=[]
    for name,digits in DIGIT_BANK.items():
        for L in range(8,3,-1):  # search len 8→4
            for i in range(len(xstr)-L+1):
                sub=xstr[i:i+L]
                if sub in digits:
                    hits.append(f'{name}:{sub}')
                    break
            if hits: break
    return ', '.join(hits)

powers=[9,27,81,127,729]

In [5]:
rows=[]
phi_centre=0.0039439
phi_vals=[phi_centre+i*2e-7 for i in range(-6,7)]  # ±1.2e-6
for bits in (64,128):
    seed=fib_fraction(bits)
    for k_tiny in (4,4.5):
        tiny=ALPHA_INV/(10**k_tiny)
        for phi in phi_vals:
            x=breath_shift(seed+tiny,tiny,phi)
            rel=float(abs((x-ALPHA)/ALPHA))
            xstr=str(x)[2:302]
            threads=digit_threads(xstr)
            pow_hit=any(abs(float(1/x)-p)/p<0.01 or abs(float(x)-p)/p<0.01 for p in powers)
            rows.append(dict(bits=bits,k=k_tiny,phi_scale=phi,rel_err=rel,threads=threads or '-',pow3=pow_hit))
df=pd.DataFrame(rows)
df.sort_values('rel_err').head(20)

,bits,k,phi_scale,rel_err,threads,pow3
35,128,4.0,0.003945,0.000064,pi:21170,False
9,64,4.0,0.003945,0.000064,pi:58700,False
8,64,4.0,0.003944,0.000082,pi:582231,False
34,128,4.0,0.003944,0.000082,pi:1058,False
36,128,4.0,0.003945,0.000210,pi:37510,False
10,64,4.0,0.003945,0.000210,pi:58223,False
7,64,4.0,0.003944,0.000228,pi:4944,False
33,128,4.0,0.003944,0.000228,pi:80348,False
37,128,4.0,0.003945,0.000356,pi:49141,False
11,64,4.0,0.003945,0.000356,pi:81748,False
